### 因子预处理(上)

In [ ]:
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

# 生成一个正态分布
def demo1():
    mu ,sigma = 0, 1
    sampleNo = 10000
    np.random.seed(0)
    s = np.random.normal(mu, sigma, sampleNo)
    plt.hist(s, bins=100, normed=True)
    plt.show()
demo1()

### 一、基础数据获取

In [1]:
start_date = '2015-01-01'
end_date='2017-08-10'
all_data = DataSource('market_value_CN_STOCK_A').read(start_date = start_date, end_date = end_date)

In [ ]:
all_data.head()

In [14]:
# 获取其中某一天数据
df = all_data[all_data['date']=='2015-01-05']

In [ ]:
# 某一天数据列的密度图分析，可以发现数据很粗糙，极值存在，需要做一些处理和转化
import seaborn as sns
sns.kdeplot(df['market_cap_float'])

### 二、观察数据分布

##### 数据右偏的话可以对所有数据取对数、取平方根等，它的原理是因为这样的变换的导数是逐渐减小的，也就是说它的增速逐渐减缓，所以就可以把大的数据向左移，使数据接近正态分布。如果左偏的话可以取相反数转化为右偏的情况。

In [ ]:
df['market_cap_float'] = np.log(df['market_cap_float'])
df['market_cap_float'].hist(bins=100, figsize=[10,6])

### 三、数据处理

#### 1. 缺失值处理

In [ ]:
# 缺失值查看
df.isnull().sum()

In [40]:
df.dropna(inplace=True)

####  2. 极值处理

In [ ]:
p_95 = df['market_cap_float'].quantile(0.95)
p_5 = df['market_cap_float'].quantile(0.05)
df_winsorize = df[(df.market_cap_float<p_95)&(df.market_cap_float>p_5)] # 截断处理
# 极值截断处理后的分布
df_winsorize['market_cap_float'].hist(bins=100, figsize=[10,6])

#### 3、标准化

In [ ]:
df_standalize = df_winsorize.copy()
df_standalize['market_cap_float'] = df_standalize[['market_cap_float']].apply(lambda x:(x-np.mean(x))/np.std(x))
df_standalize['market_cap_float'].head()

In [ ]:
df_standalize['market_cap_float'].hist(bins=100, figsize=[10,6])

## 三、整合预处理

In [55]:
# 因子预处理函数
def preprocess(df,factor):
    # 缺失值处理
    df[factor].fillna(np.nanmean(df[factor]), inplace=True)
    # 极值处理
    p_95 = df[factor].quantile(0.95)
    p_5 = df[factor].quantile(0.05)
    df = df[(df[factor] < p_95)&(df[factor] > p_5)]
    # 标准化处理
    df[factor] = (df[factor] - df[factor].mean()) / df[factor].std()
    return df[factor] 

# 按每个交易日进行因子预处理，此时因子预处理完成
df_preprocess = df.groupby('date').apply(lambda x:preprocess(x,factor='market_cap_float'))

In [ ]:
plt.figure(figsize=[10,6])
m= plt.hist(df_preprocess,bins=100)